# Scraping Dataset

In [63]:
!pip install requests beautifulsoup4
!pip install lxml
!pip install requests beautifulsoup4 pandas

You should consider upgrading via the 'C:\Users\HP 840 G8\Desktop\skripsi\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\HP 840 G8\Desktop\skripsi\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\HP 840 G8\Desktop\skripsi\venv\Scripts\python.exe -m pip install --upgrade pip' command.


## requests adalah library Python yang digunakan untuk mengirim permintaan HTTP dan menerima respons dari server. Fungsinya termasuk:

Mengambil data dari URL: Anda dapat menggunakan requests untuk mengunduh konten halaman web (HTML) atau API endpoint.

Fitur penting:

Mendukung berbagai metode HTTP: GET, POST, PUT, DELETE, dll.
Mendukung pengaturan headers, cookies, dan parameters.
Menangani koneksi HTTPS secara otomatis.

# ArXiv

In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import xml.etree.ElementTree as ET
import csv

In [68]:
import requests
import xml.etree.ElementTree as ET

def scrape_arxiv_by_date(query, start_date, end_date, total_results=5000, batch_size=100):
    base_url = "https://export.arxiv.org/api/query"
    all_articles = []
    
    # Membagi permintaan menjadi batch berdasarkan total hasil dan batch size
    for start in range(0, total_results, batch_size):
        params = {
            "search_query": f"all:{query}",
            "start": start,
            "max_results": batch_size,
            "sortBy": "relevance",
            "sortOrder": "descending"
        }
        
        # Mengirim permintaan GET
        response = requests.get(base_url, params=params)
        
        if response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            return []

        # Menggunakan xml.etree.ElementTree untuk memparsing XML
        root = ET.fromstring(response.text)
        entries = root.findall("{http://www.w3.org/2005/Atom}entry")
        
        # Mengolah artikel-artikel yang didapatkan
        for entry in entries:
            title = entry.find("{http://www.w3.org/2005/Atom}title").text.strip()
            summary = entry.find("{http://www.w3.org/2005/Atom}summary").text.strip()
            published = entry.find("{http://www.w3.org/2005/Atom}published").text.strip()
            link = entry.find("{http://www.w3.org/2005/Atom}id").text.strip()
            pdf_link = link.replace("abs", "pdf")  # Membuat link PDF dari link artikel

            # Menambahkan informasi author dan affiliation jika ada
            authors = entry.findall("{http://www.w3.org/2005/Atom}author")
            author_list = []
            institution_list = []
            for author in authors:
                name = author.find("{http://www.w3.org/2005/Atom}name")
                if name is not None:
                    author_list.append(name.text.strip())
                # Jika ada informasi institusi, dapat diambil dari elemen affiliation
                affiliation = author.find("{http://arxiv.org/schemas/atom}affiliation")
                if affiliation is not None:
                    institution_list.append(affiliation.text.strip())
            
            # Jika tidak ada affiliation yang ditemukan, bisa menambahkan "Unknown"
            if not institution_list:
                institution_list.append("Unknown")

            # Menambahkan artikel ke dalam daftar hasil
            article = {
                "Title": title,
                "Summary": summary,
                "Published": published,
                "Link": link,
                "PDF Link": pdf_link,
                "Authors": ", ".join(author_list)
            }
            all_articles.append(article)

        if len(entries) < batch_size:
            break  # Jika jumlah artikel kurang dari batch_size, hentikan permintaan lebih lanjut

    return all_articles

In [78]:
articles = scrape_arxiv_by_date(
    query="neural network OR machine learning OR AI OR artificial intelligence OR data mining", 
    start_date="2014-01-01", 
    end_date="2024-12-31", 
    total_results=10000, 
    batch_size=100
)

In [79]:
# Menampilkan hasil
import pandas as pd
df = pd.DataFrame(articles)
df

,Title,Summary,Published,Link,PDF Link,Authors
0,Behave-XAI: Deep Explainable Learning of Behav...,According to the latest trend of artificial in...,2022-12-30T18:08:48Z,http://arxiv.org/abs/2301.00016v2,http://arxiv.org/pdf/2301.00016v2,"Rossi Kamal, Zuzana Kubincova"
1,Deep Neural Mobile Networking,The next generation of mobile networks is set ...,2020-10-23T09:23:36Z,http://arxiv.org/abs/2011.05267v1,http://arxiv.org/pdf/2011.05267v1,Chaoyun Zhang
2,Learning Random Numbers to Realize Appendable ...,"In this study, we developed a learning method ...",2024-07-29T17:24:35Z,http://arxiv.org/abs/2407.20197v1,http://arxiv.org/pdf/2407.20197v1,Kazunori D Yamada
3,Modeling the EdNet Dataset with Logistic Regre...,Many of these challenges are won by neural net...,2021-05-17T20:30:36Z,http://arxiv.org/abs/2105.08150v1,http://arxiv.org/pdf/2105.08150v1,"Philip I. Pavlik Jr, Luke G. Eglington"
4,Causal Relationship Network of Risk Factors Im...,This study aims to establish the causal relati...,2024-01-24T22:45:34Z,http://arxiv.org/abs/2402.05940v1,http://arxiv.org/pdf/2402.05940v1,"Shangsi Ren, Cameron A. Beeche, Zhiyi Shi, Mar..."
...,...,...,...,...,...,...
495,Proceedings of the Pacific Knowledge Acquisiti...,Artificial intelligence (AI) research has evol...,2005-04-14T13:14:53Z,http://arxiv.org/abs/cs/0504071v1,http://arxiv.org/pdf/cs/0504071v1,"Byeong Ho Kang, Achim Hoffmann, Takahira Yamag..."
496,Fast Graph Condensation with Structure-based N...,The rapid development of Internet technology h...,2023-10-17T07:25:59Z,http://arxiv.org/abs/2310.11046v2,http://arxiv.org/pdf/2310.11046v2,"Lin Wang, Wenqi Fan, Jiatong Li, Yao Ma, Qing Li"
497,Spyx: A Library for Just-In-Time Compiled Opti...,As the role of artificial intelligence becomes...,2024-02-29T09:46:44Z,http://arxiv.org/abs/2402.18994v1,http://arxiv.org/pdf/2402.18994v1,"Kade M. Heckel, Thomas Nowotny"
498,Model-based Exception Mining for Object-Relati...,This paper is based on a previous publication ...,2018-07-01T19:42:02Z,http://arxiv.org/abs/1807.00381v1,http://arxiv.org/pdf/1807.00381v1,"Fatemeh Riahi, Oliver Schulte"


In [80]:
# Menyimpan ke CSV
df.to_csv("arxiv_dataset.csv", index=False)

## Query 2 : bert, biLSTM, information extraction

In [81]:
articles = scrape_arxiv_by_date(
    query="information extraction OR BERT OR BiLSTM", 
    start_date="2014-01-01", 
    end_date="2024-12-31", 
    total_results=10000, 
    batch_size=100
)

In [82]:
# Menampilkan hasil
import pandas as pd
df = pd.DataFrame(articles)
df

,Title,Summary,Published,Link,PDF Link,Authors
0,Clinical Trial Information Extraction with BERT,Natural language processing (NLP) of clinical ...,2021-09-11T17:15:10Z,http://arxiv.org/abs/2110.10027v1,http://arxiv.org/pdf/2110.10027v1,"Xiong Liu, Greg L. Hersch, Iya Khalil, Murthy ..."
1,Extraction of Medication and Temporal Relation...,"Clinical texts, represented in electronic medi...",2023-10-03T17:37:22Z,http://arxiv.org/abs/2310.02229v2,http://arxiv.org/pdf/2310.02229v2,"Hangyu Tu, Lifeng Han, Goran Nenadic"
2,FinBERT-MRC: financial named entity recognitio...,Financial named entity recognition (FinNER) fr...,2022-05-31T00:44:57Z,http://arxiv.org/abs/2205.15485v1,http://arxiv.org/pdf/2205.15485v1,"Yuzhe Zhang, Hong Zhang"
3,German BERT Model for Legal Named Entity Recog...,"The use of BERT, one of the most popular langu...",2023-03-07T11:54:39Z,http://arxiv.org/abs/2303.05388v1,http://arxiv.org/pdf/2303.05388v1,"Harshil Darji, Jelena Mitrović, Michael Granitzer"
4,Exploring Multimodal Sentiment Analysis via CB...,Because multimodal data contains more modal in...,2023-03-26T12:34:01Z,http://arxiv.org/abs/2303.14708v1,http://arxiv.org/pdf/2303.14708v1,"Huiru Wang, Xiuhong Li, Zenyu Ren, Dan Yang, c..."
...,...,...,...,...,...,...
1495,BERT Based Multilingual Machine Comprehension ...,Multilingual Machine Comprehension (MMC) is a ...,2020-06-02T07:36:49Z,http://arxiv.org/abs/2006.01432v1,http://arxiv.org/pdf/2006.01432v1,"Somil Gupta, Nilesh Khade"
1496,MC-BERT: Efficient Language Pre-Training via a...,"Pre-trained contextual representations (e.g., ...",2020-06-10T09:22:19Z,http://arxiv.org/abs/2006.05744v2,http://arxiv.org/pdf/2006.05744v2,"Zhenhui Xu, Linyuan Gong, Guolin Ke, Di He, Sh..."
1497,Approximate Nearest Neighbor Negative Contrast...,Conducting text retrieval in a dense learned r...,2020-07-01T23:15:56Z,http://arxiv.org/abs/2007.00808v2,http://arxiv.org/pdf/2007.00808v2,"Lee Xiong, Chenyan Xiong, Ye Li, Kwok-Fung Tan..."
1498,CLiMP: A Benchmark for Chinese Language Model ...,Linguistically informed analyses of language m...,2021-01-26T23:16:29Z,http://arxiv.org/abs/2101.11131v1,http://arxiv.org/pdf/2101.11131v1,"Beilei Xiang, Changbing Yang, Yu Li, Alex Wars..."


In [83]:
# Save to CSV
df.to_csv("arxiv(2).csv", index=False)
print("Data saved to 'arxiv(2).csv'")

Data saved to 'arxiv(2).csv'


## Query 3 : NLP, Transformers, RNN, CNN, big data

In [84]:
articles = scrape_arxiv_by_date(
    query="NLP OR RNN OR CNN OR NN OR big data OR transformers OR data structure", 
    start_date="2014-01-01", 
    end_date="2024-12-31", 
    total_results=10000, 
    batch_size=100
)

In [85]:
# Menampilkan hasil
import pandas as pd
df = pd.DataFrame(articles)
df

,Title,Summary,Published,Link,PDF Link,Authors
0,Comparison of Neural Network Architectures for...,Different neural network (NN) architectures ha...,2019-07-15T21:24:31Z,http://arxiv.org/abs/1907.07321v1,http://arxiv.org/pdf/1907.07321v1,"Ziyu Ye, Andrew Gilman, Qihang Peng, Kelly Lev..."
1,Heart Murmur and Abnormal PCG Detection via Wa...,Heart murmurs provide valuable information abo...,2023-03-12T13:49:45Z,http://arxiv.org/abs/2303.11423v2,http://arxiv.org/pdf/2303.11423v2,"Ahmed Patwa, Muhammad Mahboob Ur Rahman, Tareq..."
2,Embedding generation for text classification o...,Text classification is a natural language proc...,2022-12-01T15:24:19Z,http://arxiv.org/abs/2212.00587v1,http://arxiv.org/pdf/2212.00587v1,"Frederico Dias Souza, João Baptista de Oliveir..."
3,"RISC-NN: Use RISC, NOT CISC as Neural Network ...",Neural Networks (NN) have been proven to be po...,2021-03-23T08:49:50Z,http://arxiv.org/abs/2103.12393v1,http://arxiv.org/pdf/2103.12393v1,"Taoran Xiang, Lunkai Zhang, Shuqian An, Xiaoch..."
4,Convolutional Neural Networks for Sentiment An...,This study addressed the complex task of senti...,2023-07-13T03:02:56Z,http://arxiv.org/abs/2307.06540v1,http://arxiv.org/pdf/2307.06540v1,"Yufei Xie, Rodolfo C. Raga Jr"
...,...,...,...,...,...,...
95,DeepProtein: Deep Learning Library and Benchma...,"In recent years, deep learning has revolutioni...",2024-10-02T20:42:32Z,http://arxiv.org/abs/2410.02023v1,http://arxiv.org/pdf/2410.02023v1,"Jiaqing Xie, Yue Zhao, Tianfan Fu"
96,Crystal structure identification with 3D convo...,"Efficient, reliable and easy-to-use structure ...",2024-05-08T15:47:38Z,http://arxiv.org/abs/2405.05156v1,http://arxiv.org/pdf/2405.05156v1,"Linus C. Erhard, Daniel Utt, Arne J. Klomp, Ka..."
97,Stock Forecasting using M-Band Wavelet-Based S...,The task of predicting future stock values has...,2019-04-17T19:10:10Z,http://arxiv.org/abs/1904.08459v1,http://arxiv.org/pdf/1904.08459v1,"Hieu Quang Nguyen, Abdul Hasib Rahimyar, Xiaod..."
98,3DCNN-DQN-RNN: A Deep Reinforcement Learning F...,Semantic parsing of large-scale 3D point cloud...,2017-07-21T07:28:14Z,http://arxiv.org/abs/1707.06783v1,http://arxiv.org/pdf/1707.06783v1,"Fangyu Liu, Shuaipeng Li, Liqiang Zhang, Cheng..."


In [86]:
# Save to CSV
df.to_csv("arxiv(3).csv", index=False)
print("Data saved to 'arxiv(3).csv'")

Data saved to 'arxiv(3).csv'


## Query 4 : text mining, pattern recognition, business intelligence, data visualization 

In [87]:
articles = scrape_arxiv_by_date(
    query="business inteligence OR data visualization", 
    start_date="2014-01-01", 
    end_date="2024-12-31", 
    total_results=10000, 
    batch_size=100
)

In [88]:
# Menampilkan hasil
import pandas as pd
df = pd.DataFrame(articles)
df

,Title,Summary,Published,Link,PDF Link,Authors
0,Visualization Requirements for Business Intell...,Information visualization plays a key role in ...,2024-02-14T10:50:26Z,http://arxiv.org/abs/2402.09491v1,http://arxiv.org/pdf/2402.09491v1,"Ana Lavalle, Alejandro Maté, Juan Trujillo, St..."
1,What-if Analysis for Business Users: Current P...,"What-if analysis (WIA), crucial for making dat...",2022-12-27T23:43:02Z,http://arxiv.org/abs/2212.13643v3,http://arxiv.org/pdf/2212.13643v3,"Sneha Gathani, Zhicheng Liu, Peter J. Haas, Ça..."
2,Visualizing Multiple Process Attributes in one...,Business process models are usually visualized...,2019-03-01T13:18:12Z,http://arxiv.org/abs/1903.00283v1,http://arxiv.org/pdf/1903.00283v1,"Manuel Gall, Stefanie Rinderle-Ma"
3,Prismatic: Interactive Multi-View Cluster Anal...,Financial cluster analysis allows investors to...,2024-02-14T06:47:30Z,http://arxiv.org/abs/2402.08978v1,http://arxiv.org/pdf/2402.08978v1,"Wong Kam-Kwai, Yan Luo, Xuanwu Yue, Wei Chen, ..."
4,Extracting Similar Questions From Naturally-oc...,Pre-trained contextualized embedding models su...,2022-06-03T14:13:44Z,http://arxiv.org/abs/2206.01585v1,http://arxiv.org/pdf/2206.01585v1,"Xiliang Zhu, David Rossouw, Shayna Gardiner, S..."
...,...,...,...,...,...,...
495,Qualitative Analysis of Integration Adapter Mo...,Integration Adapters are a fundamental part of...,2015-03-06T16:36:22Z,http://arxiv.org/abs/1503.02007v1,http://arxiv.org/pdf/1503.02007v1,"Daniel Ritter, Manuel Holzleitner"
496,Powering the Internet of Things with RIOT: Why...,The crucial importance of software platforms w...,2016-03-11T14:10:18Z,http://arxiv.org/abs/1603.03635v1,http://arxiv.org/pdf/1603.03635v1,"Emmanuel Baccelli, Kaspar Schleiser"
497,Analysis of Complex System Development Based o...,This article represents one of the contemporar...,2018-02-05T12:04:54Z,http://arxiv.org/abs/1802.04109v1,http://arxiv.org/pdf/1802.04109v1,Ismail Kayali
498,Empirical effect of graph embeddings on fraud ...,Graph embedding technics are studied with inte...,2019-03-05T07:49:27Z,http://arxiv.org/abs/1903.05976v1,http://arxiv.org/pdf/1903.05976v1,Sida Zhou


In [89]:
# Save to CSV
df.to_csv("arxiv(4).csv", index=False)
print("Data saved to 'arxiv(4).csv'")

Data saved to 'arxiv(4).csv'


## Query 5 

In [90]:
articles = scrape_arxiv_by_date(
    query="orecasting OR classification OR regression OR random forest OR XGBoost", 
    start_date="2014-01-01", 
    end_date="2024-12-31", 
    total_results=10000, 
    batch_size=100
)

In [91]:
# Menampilkan hasil
import pandas as pd
df = pd.DataFrame(articles)
df

,Title,Summary,Published,Link,PDF Link,Authors
0,LinXGBoost: Extension of XGBoost to Generalize...,XGBoost is often presented as the algorithm th...,2017-10-10T14:52:47Z,http://arxiv.org/abs/1710.03634v1,http://arxiv.org/pdf/1710.03634v1,Laurent de Vito
1,Automatic pain recognition from Blood Volume P...,Physiological responses to pain have received ...,2023-03-19T09:03:14Z,http://arxiv.org/abs/2303.10607v1,http://arxiv.org/pdf/2303.10607v1,"Fatemeh Pouromran, Yingzi Lin, Sagar Kamarthi"
2,Predictive Modeling of Shading Effects on Phot...,Drones have become indispensable assets during...,2024-12-10T22:57:36Z,http://arxiv.org/abs/2412.17828v1,http://arxiv.org/pdf/2412.17828v1,"Jonathan Olivares, Tyler Depe, Kanika Sood, Ra..."
3,Predicting the duration of traffic incidents f...,This research presents a comprehensive approac...,2024-06-27T03:16:09Z,http://arxiv.org/abs/2406.18861v2,http://arxiv.org/pdf/2406.18861v2,"Artur Grigorev, Sajjad Shafiei, Hanna Grzybows..."
4,Comparison of Deep Learning and Traditional Ma...,A comprehensive study on machine and deep lear...,2020-09-11T08:20:51Z,http://arxiv.org/abs/2009.06366v1,http://arxiv.org/pdf/2009.06366v1,"Abdurrahim Yilmaz, Ali Anil Demircali, Sena Ko..."
...,...,...,...,...,...,...
995,Defending Against Adversarial Attacks Using Ra...,As deep neural networks (DNNs) have become inc...,2019-06-16T20:46:44Z,http://arxiv.org/abs/1906.06765v1,http://arxiv.org/pdf/1906.06765v1,"Yifan Ding, Liqiang Wang, Huan Zhang, Jinfeng ..."
996,FastForest: Increasing Random Forest Processin...,Random Forest remains one of Data Mining's mos...,2020-04-06T06:37:03Z,http://arxiv.org/abs/2004.02423v1,http://arxiv.org/pdf/2004.02423v1,"Darren Yates, Md Zahidul Islam"
997,Rare-Event Simulation for Neural Network and R...,We study rare-event simulation for a class of ...,2020-10-10T03:27:09Z,http://arxiv.org/abs/2010.04890v1,http://arxiv.org/pdf/2010.04890v1,"Yuanlu Bai, Zhiyuan Huang, Henry Lam, Ding Zhao"
998,Detection of Malicious Android Applications: C...,Today anti-malware community is facing challen...,2021-02-28T21:50:57Z,http://arxiv.org/abs/2103.00637v1,http://arxiv.org/pdf/2103.00637v1,"Hemant Rathore, Sanjay K. Sahay, Shivin Thukra..."


In [92]:
# Save to CSV
df.to_csv("arxiv(5).csv", index=False)
print("Data saved to 'arxiv(5).csv'")

Data saved to 'arxiv(5).csv'


## Query 6

In [93]:
articles = scrape_arxiv_by_date(
    query="data warehousing OR database", 
    start_date="2014-01-01", 
    end_date="2024-12-31", 
    total_results=10000, 
    batch_size=100
)

In [94]:
# Menampilkan hasil
import pandas as pd
df = pd.DataFrame(articles)
df

,Title,Summary,Published,Link,PDF Link,Authors
0,X-WACoDa: An XML-based approach for Warehousin...,Data warehousing and OLAP applications must no...,2017-01-27T12:39:39Z,http://arxiv.org/abs/1701.08033v1,http://arxiv.org/pdf/1701.08033v1,"Hadj Mahboubi, Jean-Christian Ralaivao, Sabine..."
1,Knowledge and Metadata Integration for Warehou...,With the ever-growing availability of so-calle...,2008-09-11T12:20:00Z,http://arxiv.org/abs/0809.1971v1,http://arxiv.org/pdf/0809.1971v1,"Jean-Christian Ralaivao, Jérôme Darmont"
2,XML Warehousing and OLAP,The aim of this article is to present an overv...,2017-01-30T14:27:07Z,http://arxiv.org/abs/1701.08612v1,http://arxiv.org/pdf/1701.08612v1,"Hadj Mahboubi, Marouane Hachicha, Jérôme Darmont"
3,XML content warehousing: Improving sociologica...,"In this paper, we present the guidelines for a...",2011-08-23T13:31:56Z,http://arxiv.org/abs/1108.4596v1,http://arxiv.org/pdf/1108.4596v1,"Benjamin Nguyen, Antoine Vion, François-Xavier..."
4,An Architecture Framework for Complex Data War...,"Nowadays, many decision support applications n...",2007-07-10T22:01:40Z,http://arxiv.org/abs/0707.1534v1,http://arxiv.org/pdf/0707.1534v1,"Jérôme Darmont, Omar Boussaid, Jean-Christian ..."
5,Warehousing Web Data,"In a data warehousing process, mastering the d...",2007-05-10T12:28:52Z,http://arxiv.org/abs/0705.1456v1,http://arxiv.org/pdf/0705.1456v1,"Jérôme Darmont, Omar Boussaïd, Fadila Bentayeb"
6,Data Warehouse Benchmarking with DWEB,Performance evaluation is a key issue for desi...,2017-01-27T13:56:37Z,http://arxiv.org/abs/1701.08053v1,http://arxiv.org/pdf/1701.08053v1,Jérôme Darmont
7,fVSS: A New Secure and Cost-Efficient Scheme f...,Cloud business intelligence is an increasingly...,2014-12-11T04:41:30Z,http://arxiv.org/abs/1412.3538v1,http://arxiv.org/pdf/1412.3538v1,"Varunya Attasena, Nouria Harbi, Jérôme Darmont"
8,Warehousing complex data from the Web,The data warehousing and OLAP technologies are...,2017-01-02T14:18:01Z,http://arxiv.org/abs/1701.00398v1,http://arxiv.org/pdf/1701.00398v1,"Omar Boussaid, Jerome Darmont, Fadila Bentayeb..."
9,Web data modeling for integration in data ware...,"In a data warehousing process, the data prepar...",2007-05-10T12:30:19Z,http://arxiv.org/abs/0705.1457v1,http://arxiv.org/pdf/0705.1457v1,"Sami Miniaoui, Jérôme Darmont, Omar Boussaïd"


In [95]:
# Save to CSV
df.to_csv("arxiv(6).csv", index=False)
print("Data saved to 'arxiv(6).csv'")

Data saved to 'arxiv(6).csv'


## Query 7

In [96]:
articles = scrape_arxiv_by_date(
    query="supervised learning OR unsupervised learning OR feature extraction", 
    start_date="2014-01-01", 
    end_date="2024-12-31", 
    total_results=10000, 
    batch_size=100
)

In [97]:
# Menampilkan hasil
import pandas as pd
df = pd.DataFrame(articles)
df

,Title,Summary,Published,Link,PDF Link,Authors
0,Unified Framework for Feature Extraction based...,Feature extraction is an efficient approach fo...,2021-01-25T16:43:03Z,http://arxiv.org/abs/2101.11703v2,http://arxiv.org/pdf/2101.11703v2,Hongjie Zhang
1,Unsupervised Feature Learning by Autoencoder a...,Unsupervised learning methods for feature extr...,2020-09-02T11:17:48Z,http://arxiv.org/abs/2009.00953v1,http://arxiv.org/pdf/2009.00953v1,"Zeyu Cao, Xiaorun Li, Liaoying Zhao"
2,Unsupervised Generative Feature Transformation...,Feature transformation is to derive a new feat...,2024-05-27T06:50:00Z,http://arxiv.org/abs/2405.16879v1,http://arxiv.org/pdf/2405.16879v1,"Wangyang Ying, Dongjie Wang, Xuanming Hu, Yuan..."
3,Progressive Stage-wise Learning for Unsupervis...,Unsupervised learning methods have recently sh...,2021-06-10T07:33:19Z,http://arxiv.org/abs/2106.05554v2,http://arxiv.org/pdf/2106.05554v2,"Zefan Li, Chenxi Liu, Alan Yuille, Bingbing Ni..."
4,A Brief Summary of Interactions Between Meta-L...,This paper briefly reviews the connections bet...,2021-03-01T08:31:28Z,http://arxiv.org/abs/2103.00845v2,http://arxiv.org/pdf/2103.00845v2,Huimin Peng
...,...,...,...,...,...,...
95,Unsupervised Spatial-Temporal Feature Enrichme...,Unsupervised skeleton based action recognition...,2024-01-25T09:24:07Z,http://arxiv.org/abs/2401.14034v1,http://arxiv.org/pdf/2401.14034v1,"Chuankun Li, Shuai Li, Yanbo Gao, Ping Chen, J..."
96,Unsupervised Vehicle Re-Identification via Sel...,The key challenge of unsupervised vehicle re-i...,2021-03-03T08:29:03Z,http://arxiv.org/abs/2103.02250v1,http://arxiv.org/pdf/2103.02250v1,"Jongmin Yu, Hyeontaek Oh"
97,Leveraging sparse and shared feature activatio...,Recovering the latent factors of variation of ...,2023-04-17T01:33:24Z,http://arxiv.org/abs/2304.07939v3,http://arxiv.org/pdf/2304.07939v3,"Marco Fumero, Florian Wenzel, Luca Zancato, Al..."
98,Unsupervised video summarization framework usi...,Video is one of the robust sources of informat...,2019-10-10T18:14:48Z,http://arxiv.org/abs/1910.04792v2,http://arxiv.org/pdf/1910.04792v2,"Shruti Jadon, Mahmood Jasim"


In [98]:
# Save to CSV
df.to_csv("arxiv(7).csv", index=False)
print("Data saved to 'arxiv(7).csv'")

Data saved to 'arxiv(7).csv'


## Query 8

In [99]:
articles = scrape_arxiv_by_date(
    query="recommendation systems OR SQL OR generative AI", 
    start_date="2014-01-01", 
    end_date="2024-12-31", 
    total_results=10000, 
    batch_size=100
)

In [100]:
# Menampilkan hasil
import pandas as pd
df = pd.DataFrame(articles)
df

,Title,Summary,Published,Link,PDF Link,Authors
0,A Survey of Large Language Model-Based Generat...,Text-to-SQL systems facilitate smooth interact...,2024-12-06T17:36:28Z,http://arxiv.org/abs/2412.05208v1,http://arxiv.org/pdf/2412.05208v1,"Aditi Singh, Akash Shetty, Abul Ehtesham, Sake..."
1,End-to-end Text-to-SQL Generation within an An...,Recent advancements in Text-to-SQL have pushed...,2024-06-17T21:33:01Z,http://arxiv.org/abs/2406.12104v1,http://arxiv.org/pdf/2406.12104v1,"Karime Maamari, Amine Mhedhbi"
2,SQLFlow: A Bridge between SQL and Machine Lear...,Industrial AI systems are mostly end-to-end ma...,2020-01-19T15:19:20Z,http://arxiv.org/abs/2001.06846v1,http://arxiv.org/pdf/2001.06846v1,"Yi Wang, Yang Yang, Weiguo Zhu, Yi Wu, Xu Yan,..."
3,EzSQL: An SQL intermediate representation for ...,The SQL-to-text generation task traditionally ...,2024-11-28T05:24:46Z,http://arxiv.org/abs/2411.18923v1,http://arxiv.org/pdf/2411.18923v1,"Meher Bhardwaj, Hrishikesh Ethari, Dennis Sing..."
4,Generative Recommendation: Towards Next-genera...,Recommender systems typically retrieve items f...,2023-04-07T07:20:16Z,http://arxiv.org/abs/2304.03516v2,http://arxiv.org/pdf/2304.03516v2,"Wenjie Wang, Xinyu Lin, Fuli Feng, Xiangnan He..."
...,...,...,...,...,...,...
95,Inspo: Writing with Crowds Alongside AI,The use of artificial intelligence (AI) to sup...,2023-11-28T05:00:18Z,http://arxiv.org/abs/2311.16521v3,http://arxiv.org/pdf/2311.16521v3,"Chieh-Yang Huang, Sanjana Gautam, Shannon McCl..."
96,"It is not ""accuracy vs. explainability"" -- we ...",We are witnessing the emergence of an AI econo...,2022-12-16T23:33:10Z,http://arxiv.org/abs/2212.11136v2,http://arxiv.org/pdf/2212.11136v2,D. Petkovic
97,Evaluating and Enhancing LLMs for Multi-turn T...,Recent advancements in large language models (...,2024-12-21T10:13:45Z,http://arxiv.org/abs/2412.17867v1,http://arxiv.org/pdf/2412.17867v1,"Ziming Guo, Chao Ma, Yinggang Sun, Tiancheng Z..."
98,Defining a Lingua Franca to Open the Black Box...,Many AI systems have a black box nature that m...,2017-09-21T22:06:26Z,http://arxiv.org/abs/1709.07528v1,http://arxiv.org/pdf/1709.07528v1,"Kenneth L. Hess, Hugo D. Paz"


In [101]:
# Save to CSV
df.to_csv("arxiv(8).csv", index=False)
print("Data saved to 'arxiv(8).csv'")

Data saved to 'arxiv(8).csv'


# Combine Datset

In [102]:
import os
import pandas as pd

In [114]:
def read_and_prepare(file):
    # Baca file CSV
    return pd.read_csv(file)

# Path ke folder CSV
path = "C:/Users/HP 840 G8/Desktop/skripsi/notebooks/arxiv dataset"

In [115]:
# Daftar semua file CSV
file_list = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.csv')]

In [116]:
# Baca semua file CSV
df_list = [read_and_prepare(file) for file in file_list]

In [117]:
# Dapatkan kolom umum
common_columns = set.intersection(*[set(df.columns) for df in df_list])

In [118]:
# Tambahkan kolom yang hilang
df_list = [df.reindex(columns=common_columns, fill_value=pd.NA) for df in df_list]

In [119]:
# Gabungkan semua data
combined_df = pd.concat(df_list, ignore_index=True)

In [120]:
# Simpan hasilnya
combined_df.to_csv('arxiv_articles_dataset.csv', index=False)
print("Dataset berhasil digabungkan dan disimpan sebagai 'arxiv_articles_dataset.csv'")

Dataset berhasil digabungkan dan disimpan sebagai 'arxiv_articles_dataset.csv'
